In [1]:
pip install -q bitsandbytes accelerate peft transformers datasets trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 23.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
pip install -U bitsandbytes 


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install newspaper3k lxml-html-clean


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 55.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 5.3 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=6ecc34a0098fae5ff5d9579abf2414b859c1819d28c3c86e4ab42aa910438094
  Stored in directory: /root/.cache/pip/wheels/fc/ab/f8/cce3a9ae6d828bd346be695f7ff54612cd22b7cbd7208d68f3
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=2ab5cfe028d199206b5db04c9ef46852faa4e91830fa47afceb2fc5d071aef23
  Stored in directory: /root/.cache/pip/wheels/80/d5/72/9cd9eccc819636436c6a6e59

In [4]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article
import json
import re
from typing import List


ARTICLE_URL_PATTERN = re.compile(
    r"https?://[^\s\"']+/\d{4}/\d{2}/\d{2}/|https?://.*(blog|news|article).*"
)

def get_blog_urls(listing_urls: List[str], num_articles: int = 500) -> List[str]:
    urls = set()
    for listing_url in listing_urls:
        print(f"🔍 Scanning {listing_url}")
        try:
            res = requests.get(listing_url, timeout=10)
            soup = BeautifulSoup(res.text, 'html.parser')
            for a in soup.find_all("a", href=True):
                href = a["href"]
                if ARTICLE_URL_PATTERN.match(href) and href not in urls:
                    urls.add(href)
                if len(urls) >= num_articles:
                    return list(urls)
        except Exception as e:
            print(f"❌ Failed to fetch {listing_url}: {e}")
    return list(urls)


def extract_article(url: str) -> dict:
    try:
        article = Article(url)
        article.download()
        article.parse()

        title = article.title.strip()
        text = article.text.strip()

        if len(title) < 10 or len(text) < 200:
            return None

        return {
            "instruction": f"Write a blog article about: {title}",
            "input": "",
            "output": {
                "title": title,
                "introduction": "", 
                "main_content": text,
                "conclusion": ""     
            }
        }
    except Exception as e:
        print(f"❌ Failed to extract {url}: {e}")
        return None


def scrape_blogs_to_jsonl(listing_urls: List[str], output_path: str, num_blogs: int = 500):
    blog_urls = get_blog_urls(listing_urls, num_blogs)
    print(f"\n✅ Found {len(blog_urls)} blog URLs.\n")

    formatted_articles = []
    for idx, url in enumerate(blog_urls):
        print(f"📄 Processing [{idx+1}/{len(blog_urls)}]: {url}")
        result = extract_article(url)
        if result:
            formatted_articles.append(result)

    with open(output_path, "w", encoding="utf-8") as f:
        for item in formatted_articles:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")

    print(f"\n✅ Saved {len(formatted_articles)} clean blogs to {output_path}")


In [5]:
LISTING_URLS = [
    "https://techcrunch.com/",
    "https://techcrunch.com/category/apps/",
    "https://techcrunch.com/category/artificial-intelligence/",
    "https://arstechnica.com/ai/",
    "https://arstechnica.com/information-technology/",
    "https://arstechnica.com/gadgets/",
    "https://www.cnet.com/ai-atlas/",
    "https://openai.com/news/",
    "https://towardsdatascience.com/",
    "https://huggingface.co/blog",
    "https://research.google/blog/",
    "https://deepmind.google/discover/blog/",
    "https://www.microsoft.com/en-us/research/blog/",
    "https://github.blog/",
    "https://venturebeat.com/",
    "https://www.zdnet.com/topic/artificial-intelligence/",
    "https://www.zdnet.com/topic/cloud/"
]
scrape_blogs_to_jsonl(
    listing_urls=LISTING_URLS,
    output_path="techblogs_tinyllama_dataset.jsonl",
    num_blogs=200
)


🔍 Scanning https://techcrunch.com/
🔍 Scanning https://techcrunch.com/category/apps/
🔍 Scanning https://techcrunch.com/category/artificial-intelligence/
🔍 Scanning https://arstechnica.com/ai/
🔍 Scanning https://arstechnica.com/information-technology/
🔍 Scanning https://arstechnica.com/gadgets/
🔍 Scanning https://www.cnet.com/ai-atlas/
🔍 Scanning https://openai.com/news/
🔍 Scanning https://towardsdatascience.com/
🔍 Scanning https://huggingface.co/blog
🔍 Scanning https://research.google/blog/
🔍 Scanning https://deepmind.google/discover/blog/
🔍 Scanning https://www.microsoft.com/en-us/research/blog/
🔍 Scanning https://github.blog/
🔍 Scanning https://venturebeat.com/
🔍 Scanning https://www.zdnet.com/topic/artificial-intelligence/
🔍 Scanning https://www.zdnet.com/topic/cloud/

✅ Found 184 blog URLs.

📄 Processing [1/184]: https://techcrunch.com/2025/07/03/meta-has-found-another-way-to-keep-you-engaged-chatbots-that-message-you-first/
📄 Processing [2/184]: https://github.blog/
📄 Processing [3

In [6]:
from datasets import load_dataset
from transformers import AutoTokenizer
from typing import Union

def load_and_format_dataset(
    path: Union[str, list],
    tokenizer_name: str,
    max_length: int = 512
):

    dataset = load_dataset("json", data_files=path, split="train")

    def format_example(example):
        output = example.get("output", {})
        return {
            "text": (
                f"<|instruction|>\n{example.get('instruction', '').strip()}\n"
                f"<|title|>\n{output.get('title', '').strip()}\n"
                f"<|main_content|>\n{output.get('main_content', '').strip()}"
            )
        }

    dataset = dataset.map(format_example)

    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    tokenizer.pad_token = tokenizer.eos_token

    # Tokenize
    def tokenize(example):
        return tokenizer(
            example["text"],
            padding="max_length",
            truncation=True,
            max_length=max_length
        )

    tokenized_dataset = dataset.map(tokenize, batched=True)
    
    return tokenized_dataset


In [7]:
# Example usage
tokenized_ds = load_and_format_dataset(
    path="techblogs_tinyllama_dataset.jsonl",
    tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    max_length=512
)

print(tokenized_ds[0]["input_ids"][:50])  # See token IDs


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/164 [00:00<?, ? examples/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Map:   0%|          | 0/164 [00:00<?, ? examples/s]

[1, 529, 29989, 2611, 4080, 29989, 29958, 13, 6113, 263, 12618, 4274, 1048, 29901, 20553, 756, 1476, 1790, 982, 304, 3013, 366, 17785, 29901, 678, 271, 29890, 1862, 393, 2643, 366, 937, 13, 29966, 29989, 3257, 29989, 29958, 13, 19346, 756, 1476, 1790, 982, 304, 3013, 366, 17785, 29901, 678]


In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM
from peft import prepare_model_for_kbit_training, get_peft_model, LoraConfig, TaskType
from transformers import BitsAndBytesConfig
import torch
import os

def train_lora_model(
    model_id: str,
    tokenized_dataset,
    output_dir: str = "./tinyllama-blog-lora",
    num_train_epochs: int = 3,
    learning_rate: float = 2e-4,
    per_device_train_batch_size: int = 1,
    gradient_accumulation_steps: int = 4,
    logging_steps: int = 50,
    save_steps: int = 100,
    save_total_limit: int = 2
):
    
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map={"": 0}
    )

    model = prepare_model_for_kbit_training(model)

    peft_config = LoraConfig(
        r=8,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type=TaskType.CAUSAL_LM,
        target_modules=["q_proj", "v_proj"]
    )

    model = get_peft_model(model, peft_config)

    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        num_train_epochs=num_train_epochs,
        logging_steps=logging_steps,
        save_steps=save_steps,
        save_total_limit=save_total_limit,
        learning_rate=learning_rate,
        fp16=True,
        optim="paged_adamw_8bit",
        report_to="none"
    )

    trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset
    )

    trainer.train()

    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    print(f"✅ Training complete. Model saved to: {output_dir}")


2025-07-13 10:20:32.653138: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752402033.036061      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752402033.140832      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
train_lora_model(
    model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    tokenized_dataset=tokenized_ds,
    output_dir="tinyllama-blog-lora",
    num_train_epochs=20  
)


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Truncating train dataset:   0%|          | 0/164 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
50,0.890000
100,0.715500
150,0.652200
200,0.672200
250,0.647800
300,0.639000
350,0.618100
400,0.612200


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

✅ Training complete. Model saved to: tinyllama-blog-lora


In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

def compare_base_and_finetuned_outputs(
    base_model_id: str,
    finetuned_model_path: str,
    prompt: str,
    max_tokens: int = 512,
    temperature: float = 0.7,
    top_p: float = 0.9,
):
    
    base_tokenizer = AutoTokenizer.from_pretrained(base_model_id)
    base_model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto")
    base_generator = pipeline("text-generation", model=base_model, tokenizer=base_tokenizer)

    finetuned_tokenizer = AutoTokenizer.from_pretrained(finetuned_model_path)
    finetuned_model = AutoModelForCausalLM.from_pretrained(finetuned_model_path, device_map="auto")
    finetuned_generator = pipeline("text-generation", model=finetuned_model, tokenizer=finetuned_tokenizer)

    base_output = base_generator(
        prompt,
        max_new_tokens=max_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        pad_token_id=base_tokenizer.eos_token_id
    )[0]["generated_text"]

    finetuned_output = finetuned_generator(
        prompt,
        max_new_tokens=max_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        pad_token_id=finetuned_tokenizer.eos_token_id
    )[0]["generated_text"]

    return {
        "base_output": base_output,
        "finetuned_output": finetuned_output
    }


In [11]:
prompt = "Write a blog article about how AI is automating supply chain and logistics.\n"

outputs = compare_base_and_finetuned_outputs(
    base_model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    finetuned_model_path="tinyllama-blog-lora",
    prompt=prompt
)

print("\n📘 Fine-tuned Output:\n", outputs["finetuned_output"])
print("\n📗 Base Model Output:\n", outputs["base_output"])


Device set to use cuda:0
Device set to use cuda:0



📘 Fine-tuned Output:
 Write a blog article about how AI is automating supply chain and logistics.
1. Introduction
2. What is AI in supply chain and logistics?
3. How are supply chain and logistics being automated?
4. What benefits are AI bringing to the industry?
5. Examples of AI in supply chain and logistics
6. What are the challenges for AI in supply chain and logistics?
7. How can businesses prepare for the future of AI in supply chain and logistics?
8. Conclusion

Introduction

The supply chain and logistics industry is undergoing a significant transformation driven by the rise of artificial intelligence (AI). With AI, businesses can automate tasks, improve efficiency, and gain insights into their supply chain operations. This article will explore how AI is automating supply chain and logistics, its benefits, and the challenges it poses for businesses.

What is AI in supply chain and logistics?

AI is a technology that enables machines to learn and make decisions based on data. I